In [ ]:
vlib = c("tidyverse", "mashr", "ashr", "ggpubr", "data.table", "tidyseurat",
         "future.apply", "arrow", 'pheatmap', "Seurat", "hdWGCNA", "magrittr")
lapply(vlib, require, character.only = TRUE, quietly = TRUE) |> suppressMessages()


base.dir = "/path/to/dir/"

setwd(paste0(base.dir, "202310-sceqtl_v6/"))

saveRDS.gz <- function(object,file,threads=4) {
  con <- pipe(paste0("pigz -p",threads," > ",file),"wb")
  saveRDS(object, file = con)
  close(con)
}
readRDS.gz <- function(file,threads=parallel::detectCores()) {
  con <- pipe(paste0("pigz -d -c -p",threads," ",file))
  object <- readRDS(file = con)
  close(con)
  return(object)
}



In [ ]:

set.seed(42)

# optionally enable multithreading
enableWGCNAThreads(nThreads = 24)

seurat_obj_full = readRDS.gz("./test_v5.RDS")

### creating full object (v6 analysis)

In [ ]:
# RNA object
dgcM = seurat_obj_full@assays$RNA@counts
md = seurat_obj_full@meta.data
seurat_obj = CreateSeuratObject(counts = dgcM, meta.data = md, assay = "RNA")

seurat_obj@reductions$harmony = seurat_obj_full@reductions$harmony
seurat_obj %>% saveRDS.gz(file = "./scRNAseq_downstream_scenic/minimal_base_obj.RNA.RDS")

In [ ]:
# (not used) SCT object

dgcM = seurat_obj_full@assays$SCT@counts
md = seurat_obj_full@meta.data
seurat_obj = CreateSeuratObject(counts = dgcM, meta.data = md, assay = "SCT")

seurat_obj@reductions$harmony = seurat_obj_full@reductions$harmony
seurat_obj %>% saveRDS.gz(file = "./scRNAseq_downstream_scenic/minimal_base_obj.RDS")

### creating subset object (v7 if needed. need to access podo)

In [ ]:
df_si = read_delim("../202404-sceQTLv7/00_sample_information_v7_240418.txt")

In [ ]:
library(tidyseurat)
seurat_obj = seurat_obj %>%
  filter(sample %in% df_si$IID) %>%
  NormalizeData() %>%
  FindVariableFeatures() %>%
  ScaleData()

seurat_obj = seurat_obj %>%
  RunPCA


seurat_obj <- SetupForWGCNA(
  seurat_obj,
  gene_select = "fraction", 
  fraction = 0.05, 
  wgcna_name = "wgcna"
)


seurat_obj <- MetacellsByGroups(
  seurat_obj = seurat_obj,
  group.by = c("anno_l1", "sample_time"), # specify the columns in seurat_obj@meta.data to group by
  reduction = 'harmony', # select the dimensionality reduction to perform KNN on
  k = 50, # nearest-neighbors parameter
  min_cells = 150,
  max_shared = 20, # maximum number of shared cells between two metacells
  ident.group = 'anno_l1', # set the Idents of the metacell seurat object
  verbose = TRUE
)

seurat_obj <- NormalizeMetacells(seurat_obj)

seurat_obj %>% saveRDS.gz(file = "../202404-sceQTLv7/04_wgcna/seurat_obj_preped_for_wgcna.n70.RNA.RDS")